# Курсовая работа: χ-распределение

В этом ноутбуке объединены все необходимые шаги:
1. Установка и настройка окружения
2. Генерация выборки и сохранение в Excel
3. Чтение выборки из Excel
4. Вычисление описательных статистик (собственные функции)
5. Смещённая и несмещённая дисперсия, выборочные моменты
6. Сравнение с pandas
7. Гистограмма с теоретической плотностью
8. Эмпирическая функция распределения


## 1. Установка окружения

В терминале выполняем
```
pip install jupyterlab pandas numpy matplotlib openpyxl
```

Запуск:
```
jupyter lab
```


In [ ]:
# %% генерация выборки и сохранение в Excel
import numpy as np
import pandas as pd

# Параметры выборки
k = 5  # число степеней свободы
n = 100  # размер выборки

# Генерация выборки χ-распределения
data = np.sqrt((np.random.randn(n, k) ** 2).sum(axis=1))
df = pd.DataFrame({'chi_values': data})

# Сохранение в Excel
df.to_excel('chi_sample.xlsx', index=False)
print('Файл chi_sample.xlsx сохранён.')

In [ ]:
# %% чтение выборки из Excel
import pandas as pd

df = pd.read_excel('chi_sample.xlsx', engine='openpyxl')
print('Колонки:', df.columns.tolist())
chi_sample = df['chi_values']
print(f"Размер выборки: {len(chi_sample)}")
display(chi_sample.head())

In [ ]:
# %% собственные функции для описательных статистик
from collections import Counter

def sample_sum(data):
    total = 0.0
    for x in data:
        total += x
    return total

def sample_mean(data):
    return sample_sum(data) / len(data)

def sample_median(data):
    sorted_data = sorted(data)
    n = len(sorted_data)
    mid = n // 2
    if n % 2 == 1:
        return sorted_data[mid]
    else:
        return 0.5 * (sorted_data[mid - 1] + sorted_data[mid])

def sample_mode(data):
    counts = Counter(data)
    max_count = max(counts.values())
    return [k for k, v in counts.items() if v == max_count]

def sample_range(data):
    return max(data) - min(data)

def sample_variance(data):
    mu = sample_mean(data)
    s = sum((x - mu)**2 for x in data)
    return s / (len(data) - 1)

def central_moment(data, r):
    mu = sample_mean(data)
    m = sum((x - mu)**r for x in data)
    return m / len(data)


In [ ]:
# %% проверка функций
data_list = chi_sample.tolist()
print("Сумма:", sample_sum(data_list))
print("Среднее:", sample_mean(data_list))
print("Медиана:", sample_median(data_list))
print("Мода:", sample_mode(data_list))
print("Размах:", sample_range(data_list))
print("Дисперсия (несмещённая):", sample_variance(data_list))
print("2-й центральный момент:", central_moment(data_list, 2))
print("3-й центральный момент:", central_moment(data_list, 3))

In [ ]:
# %% смещённая и несмещённая дисперсия, выборочный момент
def variance_biased(data):
    mu = sample_mean(data)
    s2 = sum((x - mu)**2 for x in data) / len(data)
    return s2

def variance_unbiased(data):
    mu = sample_mean(data)
    s2 = sum((x - mu)**2 for x in data) / (len(data) - 1)
    return s2

def sample_moment_order(data, k):
    return sum(x**k for x in data) / len(data)

print("Смещённая дисперсия:", variance_biased(data_list))
print("Несмещённая дисперсия:", variance_unbiased(data_list))
print("Выборочный 1-й момент:", sample_moment_order(data_list, 1))
print("Выборочный 2-й момент:", sample_moment_order(data_list, 2))

In [ ]:
import matplotlib.pyplot as plt
from math import gamma

k = 5
data_arr = chi_sample.to_numpy()

# Гистограмма
plt.hist(data_arr, bins=15, density=True, alpha=0.6, edgecolor='black')

# Теоретическая плотность
x = np.linspace(0, data_arr.max()+1, 200)
coef = 2**(1 - k/2) / gamma(k/2)
pdf = coef * x**(k - 1) * np.exp(-x**2 / 2)
plt.plot(x, pdf, linewidth=2, label=f'χ-PDF (k={k})')

plt.title('Гистограмма и теоретическая плотность')
plt.xlabel('Значение')
plt.ylabel('Плотность')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# %% эмпирическая функция распределения
import matplotlib.pyplot as plt

sorted_data = np.sort(data_arr)
n = len(sorted_data)
y = np.arange(1, n+1) / n
plt.step(sorted_data, y, where='post', label='ЭФР')
plt.title('Эмпирическая функция распределения')
plt.xlabel('Значение')
plt.ylabel('F̂(x)')
plt.grid(True)
plt.legend()
plt.show()